In [2]:
import pandas as pd
import altair as alt
import eco_style
alt.themes.enable("light")

ThemeRegistry.enable('light')

In [35]:
image = alt.Chart(pd.DataFrame([{'url': 'https://raw.githubusercontent.com/EconomicsObservatory/ECOvisualisations/refs/heads/main/guidelines/logos/eco-logo-light.png'}])).mark_image(
    height=30,
    width=90,
    opacity=0.5,
    # align='right',
    baseline='bottom'
).encode(
    url='url',
    x=alt.value(400),
    y=alt.value(0)
)

# Chart: UK top Exports to US

In [55]:
df = pd.read_excel("data/news_TradeData.xlsx", sheet_name="Top20")

rebinds = {
  "Road vehicles (including air-cushion vehicles)": "Road Vehicles",
  "Medicinal and pharmaceutical products": "Medicinal and Pharmaceutical",
  "Power-generating machinery and equipment": "Power-Generating Equipment",
  "Miscellaneous manufactured articles, n.e.s.": "Misc. Manufactured Articles",
  "General industrial machinery and equipment, n.e.s., and machine parts, n.e.s.": "Industrial Machinery and Parts",
  "Professional, scientific and controlling instruments and apparatus, n.e.s.": "Scientific and Control Instruments",
  "Electrical machinery, apparatus and appliances, n.e.s., and electrical parts thereof (including non-electrical counterparts, n.e.s., of electrical household-type equipment)": "Electrical Machinery and Parts",
  "Other transport equipment": "Other Transport Equipment",
  "Machinery specialized for particular industries": "Specialised Industrial Machinery",
  "Gold, non-monetary (excluding gold ores and concentrates)": "Gold (Non-Monetary)",
  "Non-ferrous metals": "Non-Ferrous Metals",
  "Beverages": "Beverages",
  "Organic chemicals": "Organic Chemicals",
  "Special transactions and commodities not classified according to kind": "Special Transactions and Commodities",
  "Chemical materials and products, n.e.s.": "Chemical Materials",
  "Manufactures of metals, n.e.s.": "Metal Manufactures",
  "Telecommunications and sound-recording and reproducing apparatus and equipment": "Telecom and Audio Equipment",
  "Petroleum, petroleum products and related materials": "Petroleum and Related Products",
  "Furniture and parts thereof; bedding, mattresses, mattress supports, cushions and similar stuffed furnishings": "Furniture and Bedding",
  "Office machines and automatic data-processing machines": "Office and Data Machines"
}

df['shortProduct'] = df['Type'].replace(rebinds)
df['Value'] = df['Value'].astype(float)/1000000000

df['order'] = df['Value'].rank(method='first', ascending=False)

base = alt.Chart(df).encode(
    x=alt.X('Value:Q', title='', axis=alt.Axis(format="$,.0f", labelExpr="datum.label + 'B'")),
    y=alt.Y('shortProduct:N', title='', sort="-x"),
    order=alt.Order('order:Q', sort='ascending'),
).properties(
    width=400,
    height=500
)

labels = alt.Chart(df).mark_text(
    align='left',
    baseline='middle',
    dx=3,
    fontSize=12
).encode(
    x=alt.value(0),
    y=alt.Y('shortProduct:N', title=''),
    text=alt.Text('shortProduct:N'),
)

bars = base.mark_bar(color="#36b7b4")

chart = bars + image.encode(x=alt.value(335), y=alt.value(-10)) 

chart.save("charts/top20_rd_division.png", scale_factor=3)
chart.save("charts/top20_rd_division.json", scale_factor=3)
chart


/Users/finn/anaconda3/envs/workshop/lib/python3.11/site-packages/altair/utils/core.py:395: FutureWarning: the convert_dtype parameter is deprecated and will be removed in a future version.  Do ``ser.astype(object).apply()`` instead if you want ``convert_dtype=False``.
  col = df[col_name].apply(to_list_if_array, convert_dtype=False)
/Users/finn/anaconda3/envs/workshop/lib/python3.11/site-packages/altair/utils/core.py:395: FutureWarning: the convert_dtype parameter is deprecated and will be removed in a future version.  Do ``ser.astype(object).apply()`` instead if you want ``convert_dtype=False``.
  col = df[col_name].apply(to_list_if_array, convert_dtype=False)
/Users/finn/anaconda3/envs/workshop/lib/python3.11/site-packages/altair/utils/core.py:395: FutureWarning: the convert_dtype parameter is deprecated and will be removed in a future version.  Do ``ser.astype(object).apply()`` instead if you want ``convert_dtype=False``.
  col = df[col_name].apply(to_list_if_array, convert_dtype=Fa

alt.LayerChart(...)

# Chart: SITC level 1

In [103]:
df = pd.read_excel("data/news_TradeData.xlsx", sheet_name="SITC, Level 1")
df.rename(columns={'cmdDesc': 'Type', 'primaryValue': 'Value'}, inplace=True)
df = df[['Type', 'Value']]
df['shortProduct'] = df['Type'].replace(rebinds)
df['class'] = df['Type']

df['order'] = df['Value'].rank(method='first', ascending=False)

# disaggregate Vehicles
road_vehicle_exports = 11_454_448_037 # from SITC, Level 2
# subtract it from  Machinery and transport equipment
df.loc[df['Type'] == "Machinery and transport equipment", 'Value'] -= road_vehicle_exports

df = pd.concat([df, pd.DataFrame([{
    'Type': "Road Vehicles",
    'Value': road_vehicle_exports,
    'shortProduct': "Road Vehicles",
    'class': "Machinery and transport equipment",
    'order': 1
}])], ignore_index=True)

df = df.sort_values(by='order', ascending=True)

rebinds = {
  "Machinery and transport equipment": "Machinery & Transport",
  "Road Vehicles": "Road Vehicles",
  "Chemicals and related products, n.e.s.": "Chemicals & Related Products",
  "Miscellaneous manufactured articles": "Misc. Manufactured Articles",
  "Manufactured goods classified chiefly by material": "Manufactured Goods (by Material)",
  "Commodities and transactions not classified elsewhere in the SITC": "Other Commodities & Transactions",
  "Beverages and tobacco": "Beverages & Tobacco",
  "Food and live animals": "Food & Live Animals",
  "Mineral fuels, lubricants and related materials": "Fuels & Lubricants",
  "Crude materials, inedible, except fuels": "Crude Materials (Excl. Fuels)",
  "Animal and vegetable oils, fats and waxes": "Animal & Veg Oils/Fats/Waxes"
}

df['class'] = df['class'].replace(rebinds)

df['Value'] = df['Value'].astype(float)/1000000000


base = alt.Chart(df).encode(
    x=alt.X('Value:Q', title='', axis=alt.Axis(format="$,.0f", labelExpr="datum.label + 'B'")),
    y=alt.Y('class:N', title='', sort="-x"),
    color=alt.condition(
        "datum.shortProduct == 'Road Vehicles'",
        alt.value("#a4e7e6"),  # Road Vehicles
        alt.value("#36b7b4")  # Other products
    ),
    order=alt.Order('order:Q', sort='ascending'),
).properties(
    width=400,
    height=500
)

bars = base.mark_bar(color="#36b7b4")

# label the road vehicles
road_vehicle_label = alt.Chart(df[df['shortProduct'] == 'Road Vehicles']).mark_text(
    align='center',
    baseline='middle',
    dx=3,
    color="grey",
    opacity=0.8,
    fontSize=12
).encode(
    x=alt.value(330),
    y=alt.value(25),
    text=alt.Text('shortProduct:N'),
)

chart = bars  +road_vehicle_label+ image.encode(x=alt.value(335), y=alt.value(-10)) 

chart.save("charts/top20_sitc1_division_with_vehicles.png", scale_factor=3)
chart.save("charts/top20_sitc1_division_with_vehicles.json", scale_factor=3)
chart

/Users/finn/anaconda3/envs/workshop/lib/python3.11/site-packages/altair/utils/core.py:395: FutureWarning: the convert_dtype parameter is deprecated and will be removed in a future version.  Do ``ser.astype(object).apply()`` instead if you want ``convert_dtype=False``.
  col = df[col_name].apply(to_list_if_array, convert_dtype=False)
/Users/finn/anaconda3/envs/workshop/lib/python3.11/site-packages/altair/utils/core.py:395: FutureWarning: the convert_dtype parameter is deprecated and will be removed in a future version.  Do ``ser.astype(object).apply()`` instead if you want ``convert_dtype=False``.
  col = df[col_name].apply(to_list_if_array, convert_dtype=False)
/Users/finn/anaconda3/envs/workshop/lib/python3.11/site-packages/altair/utils/core.py:395: FutureWarning: the convert_dtype parameter is deprecated and will be removed in a future version.  Do ``ser.astype(object).apply()`` instead if you want ``convert_dtype=False``.
  col = df[col_name].apply(to_list_if_array, convert_dtype=Fa

alt.LayerChart(...)

In [94]:
df.Type.unique()

array(['Machinery and transport equipment', 'Road Vehicles',
       'Chemicals and related products, n.e.s.',
       'Miscellaneous manufactured articles',
       'Manufactured goods classified chiefly by material',
       'Commodities and transactions not classified elsewhere in the SITC',
       'Beverages and tobacco', 'Food and live animals',
       'Mineral fuels, lubricants and related materials',
       'Crude materials, inedible, except fuels',
       'Animal and vegetable oils, fats and waxes'], dtype=object)

# Chart: Modes of Transport

In [56]:
df = pd.read_excel("data/news_TradeData.xlsx", sheet_name="Transport")
df.columns = ['mode', 'value']

df['value'] = df['value'].astype(float)/1000000000

df['share'] = df['value'] / df['value'].sum()
df = df.iloc[:3]
other_share = 1-df.iloc[:2]['share'].sum()
other_share
df.iloc[2, 2] = other_share
df



base = alt.Chart(df).encode(
    x=alt.X('value:Q', title='', axis=alt.Axis(format="$,.0f", labelExpr="datum.label + 'B'")),
    y=alt.Y('mode:N', title='', sort="-x"),
).properties(
    width=400,
    height=200
)

labels = alt.Chart(df).mark_text(
    align='left',
    baseline='middle',
    dx=3,
    fontSize=12
).encode(
    x=alt.value(0),
    y=alt.Y('mode:N', title=''),
    text=alt.Text('mode:N'),
)

bars = base.mark_bar(color="#36b7b4")

bars + labels

chart = bars + image.encode(x=alt.value(335), y=alt.value(-10))

chart.save("charts/top3_transport.png", scale_factor=3)
chart.save("charts/top3_transport.json", scale_factor=3)
chart

/Users/finn/anaconda3/envs/workshop/lib/python3.11/site-packages/altair/utils/core.py:395: FutureWarning: the convert_dtype parameter is deprecated and will be removed in a future version.  Do ``ser.astype(object).apply()`` instead if you want ``convert_dtype=False``.
  col = df[col_name].apply(to_list_if_array, convert_dtype=False)
/Users/finn/anaconda3/envs/workshop/lib/python3.11/site-packages/altair/utils/core.py:395: FutureWarning: the convert_dtype parameter is deprecated and will be removed in a future version.  Do ``ser.astype(object).apply()`` instead if you want ``convert_dtype=False``.
  col = df[col_name].apply(to_list_if_array, convert_dtype=False)
/Users/finn/anaconda3/envs/workshop/lib/python3.11/site-packages/altair/utils/core.py:395: FutureWarning: the convert_dtype parameter is deprecated and will be removed in a future version.  Do ``ser.astype(object).apply()`` instead if you want ``convert_dtype=False``.
  col = df[col_name].apply(to_list_if_array, convert_dtype=Fa

alt.LayerChart(...)